In [12]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

In [3]:
# 중복된 피처명을 수정하는 함수
def get_new_feature_name_df(old_feature_name_df):
    # 중복된 피처명을 'dup_cnt' 컬럼으로 가지는 데이터프레임 생성
    feature_dup_df = pd.DataFrame(data=old_feature_name_df.groupby('column_name').cumcount(),
                                  columns=['dup_cnt'])
    feature_dup_df = feature_dup_df.reset_index()
    
    # 기존 피처명 데이터프레임과 중복 피처명 데이터프레임을 병합
    new_feature_name_df = pd.merge(old_feature_name_df.reset_index(), feature_dup_df, how='outer')
    
    # 중복된 피처명에 대해 '_dup_cnt'를 추가하여 새로운 피처명을 생성
    new_feature_name_df['column_name'] = new_feature_name_df[['column_name', 'dup_cnt']].apply(
        lambda x: x[0] + '_' + str(x[1]) if x[1] > 0 else x[0], axis=1)
    
    # 'index' 컬럼 제거하여 새로운 피처명 데이터프레임 반환
    new_feature_name_df = new_feature_name_df.drop(['index'], axis=1)
    
    return new_feature_name_df

# 휴먼 액티비티 인식 데이터셋 로드 및 전처리 함수
def get_human_dataset():
    # 피처명을 포함한 데이터 파일 로드
    feature_name_df = pd.read_csv('../dataset/UCI_HAR_Dataset/features.txt', sep='\s+',
                                  header=None, names=['column_index', 'column_name'])
    
    # 중복된 피처명을 수정하는 함수를 호출하여 새로운 피처명 데이터프레임 생성
    new_feature_name_df = get_new_feature_name_df(feature_name_df)
    
    # 새로운 피처명을 리스트로 변환
    feature_name = new_feature_name_df.iloc[:, 1].values.tolist()
    
    # 학습 데이터와 테스트 데이터 로드. 컬럼명에 피처명 적용
    X_train = pd.read_csv('../dataset/UCI_HAR_Dataset/train/X_train.txt', sep='\s+', names=feature_name)
    X_test = pd.read_csv('../dataset/UCI_HAR_Dataset/test/X_test.txt', sep='\s+', names=feature_name)
    
    # 학습 레이블과 테스트 레이블 데이터 로드. 컬럼명을 'action'으로 부여
    y_train = pd.read_csv('../dataset/UCI_HAR_Dataset/train/y_train.txt', sep='\s+', header=None, names=['action'])
    y_test = pd.read_csv('../dataset/UCI_HAR_Dataset/test/y_test.txt', sep='\s+', header=None, names=['action'])
    
    # 로드한 학습 데이터와 테스트 데이터를 모두 반환
    return X_train, X_test, y_train, y_test

In [5]:
# get_human_dataset()를 사용해 Train/Test data load
X_train, X_test, y_train, y_test = get_human_dataset()

# 학습 피처 데이터의 정보 출력
print("## 학습 피처 데이터 정보 ##")
print(X_train.info())

## 학습 피처 데이터 정보 ##
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7352 entries, 0 to 7351
Columns: 561 entries, tBodyAcc-mean()-X to angle(Z,gravityMean)
dtypes: float64(561)
memory usage: 31.5 MB
None


# RF

In [14]:
# 랜덤 포레스트 분류 모델 생성 및 학습
rf_clf = RandomForestClassifier(random_state=0)
rf_clf.fit(X_train, y_train)

# 학습한 모델을 사용하여 테스트 데이터에 대한 예측 수행
pred = rf_clf.predict(X_test)

# 예측 성능 평가를 위해 정확도를 계산하고 출력
accuracy = accuracy_score(y_test, pred)

# 예측 결과 및 평가 지표 출력
print("\n--- Random Forest ---")
print("Accuracy:", accuracy_score(y_test, pred))  # 정확도 출력
print("Confusion Matrix:\n", confusion_matrix(y_test, pred))  # 혼동 행렬 출력
print('랜덤 포레스트 정확도: {0:.4f}'.format(accuracy))  # 정확도 출력


--- Random Forest ---
Accuracy: 0.9253478113335596
Confusion Matrix:
 [[486   4   6   0   0   0]
 [ 38 426   7   0   0   0]
 [ 22  44 354   0   0   0]
 [  0   0   0 440  51   0]
 [  0   0   0  48 484   0]
 [  0   0   0   0   0 537]]
랜덤 포레스트 정확도: 0.9253


# LR

In [15]:
# 로지스틱 회귀 분류 모델 생성 및 학습
clf_lr = LogisticRegression(random_state=0)
clf_lr.fit(X_train, y_train)

# 학습한 모델을 사용하여 테스트 데이터에 대한 예측 수행
pred_lr = clf_lr.predict(X_test)

# 예측 성능 평가를 위해 정확도를 계산하고 출력
accuracy = accuracy_score(y_test, pred_lr)

# 예측 결과 및 평가 지표 출력
print("\n--- Logistic Regression Classifier ---")
print("Accuracy:", accuracy_score(y_test, pred_lr))  # 정확도 출력
print("Confusion Matrix:\n", confusion_matrix(y_test, pred_lr))  # 혼동 행렬 출력
print('Logistic Regression 정확도: {0:.4f}'.format(accuracy))  # 정확도 출력


--- Logistic Regression Classifier ---
Accuracy: 0.9579233118425518
Confusion Matrix:
 [[492   1   3   0   0   0]
 [ 24 445   2   0   0   0]
 [  4  14 402   0   0   0]
 [  0   3   0 431  57   0]
 [  0   0   0  16 516   0]
 [  0   0   0   0   0 537]]
Logistic Regression 정확도: 0.9579


# DT

In [16]:
# 의사 결정 트리 분류 모델 생성 및 학습
clf_dt = DecisionTreeClassifier(random_state=0)
clf_dt.fit(X_train, y_train)

# 학습한 모델을 사용하여 테스트 데이터에 대한 예측 수행
pred_dt = clf_dt.predict(X_test)

# 예측 성능 평가를 위해 정확도를 계산하고 출력
accuracy = accuracy_score(y_test, pred_dt)

# 예측 결과 및 평가 지표 출력
print("\n--- Decision Tree Classifier ---")
print("Accuracy:", accuracy_score(y_test, pred_dt))  # 정확도 출력
print("Confusion Matrix:\n", confusion_matrix(y_test, pred_dt))  # 혼동 행렬 출력
print('Decision Tree Classifier 정확도: {0:.4f}'.format(accuracy))  # 정확도 출력


--- Decision Tree Classifier ---
Accuracy: 0.8595181540549711
Confusion Matrix:
 [[448  24  24   0   0   0]
 [ 74 367  30   0   0   0]
 [ 23  46 351   0   0   0]
 [  0   0   0 373 118   0]
 [  0   0   0  75 457   0]
 [  0   0   0   0   0 537]]
Decision Tree Classifier 정확도: 0.8595


# SVM

In [17]:
# SVM 분류 모델 생성 및 학습
clf_svm = SVC(random_state=0)
clf_svm.fit(X_train, y_train)

# 학습한 모델을 사용하여 테스트 데이터에 대한 예측 수행
pred_svm = clf_svm.predict(X_test)

# 예측 성능 평가를 위해 정확도를 계산하고 출력
accuracy = accuracy_score(y_test, pred_svm)

# 예측 결과 및 평가 지표 출력
print("\n--- SVM Classifier ---")
print("Accuracy:", accuracy_score(y_test, pred_svm))  # 정확도 출력
print("Confusion Matrix:\n", confusion_matrix(y_test, pred_svm))  # 혼동 행렬 출력
print('SVM 정확도: {0:.4f}'.format(accuracy))  # 정확도 출력


--- SVM Classifier ---
Accuracy: 0.9504580929759077
Confusion Matrix:
 [[488   5   3   0   0   0]
 [ 20 451   0   0   0   0]
 [ 10  26 384   0   0   0]
 [  0   2   0 438  51   0]
 [  0   0   0  29 503   0]
 [  0   0   0   0   0 537]]
SVM 정확도: 0.9505
